We became familiar with several ML techniques for Classification. 

In this activity we will use Artificial Neural Networks (ANN) for classification and compare it with a logistic regression classifier.

# Dataset

We will use the ***BankCustomers_ChurnModeling*** dataset.
Download the dataset from elearn and upload it to your colab environment (on the left bar under "Files", click on Upload and browse to where you downloaded the file - as we did before).  

In [ ]:
#import dataset 
import pandas as pd
df = pd.read_csv('BankCustomers_ChurnModeling.csv') 
#defining input and outcome variables
y = df[['LeftBank']]  # y=Whether the customer left the bank (1) or not (0)
X = df.iloc[:, 3:13] # X=all other variables except for ID, Surname, row
df.head(2)

The outcome we want to predict is the binary variable "LeftBank"; which indicates whether the customer has left the bank (1) or not (0).

Hence, we want to build a binary Classifier

In [ ]:
#RUN CELL ONCE ONLY! otherwise start from above
#dummy encoding Geography and Gender variables 
X=pd.concat([X,pd.get_dummies(X[['Geography','Gender']])],axis=1) 
# drop the Original vars, since we now have the binary encoded vars
X.drop(X[['Geography','Gender']],axis=1,inplace=True)
X.head(2)

## Scaling the data

In [ ]:
# Feature Scaling, using MinMax scaling
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc = MinMaxScaler()
X_sc=pd.DataFrame(sc.fit_transform(X))
#Note: MinMax scaling Y does not make a difference since it is binary (0/1)

# Training a Logit Classifier using a train/test split

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
#uncomment ONLY one of the below lines
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123) #train/test split with unscaled data
#X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size = 0.2, random_state = 123) #with scaled data

To answer Question1: 
In the above cell use X_sc (2nd line) instead of X to create the testing and training sets with the scaled data. Then rerun the following cells till end of model performance (before k-fold CV).

In [ ]:
from sklearn.linear_model import LogisticRegression
log1 = LogisticRegression(random_state = 0, solver='liblinear')
log1.fit(X_train, y_train)

## Evaluating model performance

Evaluate the model on the testing data.

In [ ]:
!pip install scikit-plot  #run once 

In [ ]:
#evaluating the Logistic Classifier on X_test
import sklearn.metrics as metrics
import scikitplot as skplt
y_pred = log1.predict(X_test)
#confusion matrix for the DecTree Classifier
skplt.metrics.plot_confusion_matrix(y_test, y_pred,normalize=False) #set normalize=True for percentages
#calculating other model evalution metrics
print('Precision: '+ str(metrics.precision_score(y_test, y_pred))) #Precision
print('Recall:    ' + str(metrics.recall_score(y_test, y_pred))) #Recall
print('Accuracy:  ' + str(metrics.accuracy_score(y_test, y_pred)))#Accuracy
print('Balanced Acc.: ' + str(metrics.accuracy_score(y_test, y_pred,normalize=True))) # Balanced Accuracy (useful for unbalanced outcome class)

### Question1
Retrain the classifier on the scaled dataset (see above note). 
Does the model's predictive performance improve? 
...

If so, for which of the three metrics? 
...

## Plotting ROC and Precision-Recall Curves (LogisticReg)

In [ ]:
probas = log1.predict_proba(X_test)
#ROC curve 
skplt.metrics.plot_roc(y_test, probas)
#Precision-Recall curve 
skplt.metrics.plot_precision_recall(y_test, probas)

## Training and evaluating several classification models using k-fold CV

This block is the same we used in activity **2 on Classification**.

We train several classifiers using K-fold CV to improve the metric specified for "scoring=" (in the previous activity we repeated it for precision, recall, and accuracy).

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# Compare Algorithms
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# prepare models
models = []
models.append(('LogisticReg', LogisticRegression(max_iter=500)))
models.append(('K-NN      ', KNeighborsClassifier()))
models.append(('DecisTree', DecisionTreeClassifier(criterion = 'entropy')))
models.append(('NaiveBayes', GaussianNB()))
models.append(('SVM       ', SVC()))
#models.append(('RandForest', RandomForestClassifier(n_estimators = 10,criterion = 'entropy')))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy' #metric we want to compare ('accuracy')
#see https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter for complete list of options
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=123)
 	#Remember, in the next line we specify what data to use X or X_sc (the rest does not change)
	cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring) 
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison: ' +scoring)
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
#re-running on scaled dataset

import warnings
warnings.filterwarnings('ignore')
# Compare Algorithms
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# prepare models
models = []
models.append(('LogisticReg', LogisticRegression(max_iter=500)))
models.append(('K-NN      ', KNeighborsClassifier()))
models.append(('DecisTree', DecisionTreeClassifier(criterion = 'entropy')))
models.append(('NaiveBayes', GaussianNB()))
models.append(('SVM       ', SVC()))
#models.append(('RandForest', RandomForestClassifier(n_estimators = 10,criterion = 'entropy')))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy' #metric we want to compare ('accuracy')
#see https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter for complete list of options
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=123)
 	#Remember, in the next line we specify what data to use X or X_sc (the rest does not change)
	cv_results = model_selection.cross_val_score(model, X_sc, y, cv=kfold, scoring=scoring) 
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison: ' +scoring)
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

### Question2
Why is the performance of the SVM classifier so low? 

Does training the models on the scaled data fix the issue?


### Question 3
Which model has the highest Accuracy when you (correctly) train it on the scaled dataset? 

# Using a Neural Network for Classification (predict Customer churn) 

Let's use an Artificial Neural Network (ANN) for the same problem.

In [ ]:
#install some more things we need 
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
#%tensorflow_version 2.x
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# just to make sure we have the right train/test splits before we proceed
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size = 0.2, random_state = 123) #train/test split with unscaled data
#Our predictor variables
X_train.head(1)

## Setting up the NN architecture
We define the following architecture for our ANN in the next cell (and make it a function):
* 1 input layer with 13 nodes; equal to the number of predictor/input variables (see X_train above)
* 2 fully-connected (dense) hidden layer with 32 nodes; we use the Rectifier Linear (ReLu) activation function for the nodes in the hiddern layer. 
* 1 output node (we have a binary outcome) with a **Sigmoid** activation function.

Later, we will add another hidder layer

In [ ]:
len(X_train.keys())

In [ ]:
#NOTE:run this cell if you want to delete the model and clear tf session
tf.keras.backend.clear_session()
#del model

In [ ]:
def build_model():
  model = keras.Sequential([ #keras.Sequential creates a stack of layers (like a placeholder for layers we want to add)
    layers.Dense(32, activation='relu', input_shape=[len(X_train.keys())]), 
    layers.Dense(32, activation='relu'), #uncomment this line to add a 2nd hidden layer later
    layers.Dense(1,activation='sigmoid')
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.002) #https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

  #using cross-entropy as loss function (high when there are more misclassifications between true and predicted labels)
  model.compile(loss='binary_crossentropy', # https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy
                optimizer=optimizer, 
                metrics=['accuracy','AUC','TrueNegatives','TruePositives']) # https://www.tensorflow.org/api_docs/python/tf/keras/metrics 
  return model

#see the links for more details on loss, metrics, and optimizer options

In [ ]:
#call the above function once to create our model (just initializing it)
model=build_model() 
# show model architecture (see below for details)
model.summary() 

In [ ]:
model.weights

For the above NN architecture the number of parameters that can be trained are 1537. How so? 
we have 1 parameter for each weight and 1 bias parameter for each node (except for our input nodes)
*   13x32=416 (weights connecting each input node to each node in the hidden layer) +
*   32 (bias parameter for nodes in the 1st hidden layer) 
*   = 448 parameters
*   32x32=1024 (weights connecting each node in the 1st hidden layer to 32 nodes in the 2nd hidden layer) +
*   32 (bias parameters for nodes in the 2nd hidden layer) = 1056 parameters
*   = 1056
*   32 (weights connecting 32 nodes of the 2nd hidden layer to the ouput node) + 1 (bias for ouput node)
*   = 33


In [ ]:
#making sure our X_train has the right shape and we it to our NN
model.predict((X_train[:5]))

## Training the NN model 

In [ ]:
# Feature Scaling, we will use MinMax scaling
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc = MinMaxScaler() 
#sc.fit_transform(X_train)

In [ ]:
#training the NN on scaled data (NOTE: this is the right way!)
EPOCHS = 100
history = model.fit(
  X_train,y_train,
  epochs=EPOCHS, validation_split = 0.1, verbose=0, #NN trains on the other 90%, validates on 10% in each epoch
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
#take a look at the loss values for the first and last 5 epochs
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.head(5), hist.tail(5)

Our loss here is "binary cross-entropy" (lower is better). 
But we will plot Accuracy since you are more familiar with this metrics

In [ ]:
#plotting accuracy values per epoch; shows how accuracy increases as we train the NN
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "accuracy")
plt.ylim([0, 1]) 
plt.ylabel('Accuracy')

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "loss")
plt.ylim([0, 1]) 
plt.ylabel('binary cross-entropy')

### Question 4
Does the NN model training converge? If so, around which epoch?




What metric do you look at to answer the above question?

## NN model performance 

In [ ]:
#using the neural network to predict probability of outcome=1 for X_test
model.predict(X_test)
# we consider an obs with a probability of greater&equal than 0.5 to belong to the 1 class
y_pred = model.predict(X_test)[:,0]>=0.5


In [ ]:
import sklearn.metrics as metrics
import scikitplot as skplt
#confusion matrix for the DecTree Classifier
skplt.metrics.plot_confusion_matrix(y_test, y_pred,normalize=False) #set normalize=True for percentages
#calculating other model evalution metrics
print('Precision: '+ str(metrics.precision_score(y_test, y_pred))) #Precision
print('Recall:    ' + str(metrics.recall_score(y_test, y_pred))) #Recall
print('Accuracy:  ' + str(metrics.accuracy_score(y_test, y_pred)))#Accuracy
print('Balanced Acc.: ' + str(metrics.accuracy_score(y_test, y_pred,normalize=True)))

### Question 5
Does this NN model (13-32-32-1 architecture) have a better Accuracy than the previous classifiers?


## Plotting ROC and Precision-Recall Curves (NN model)

In [ ]:
import numpy as np
c1=1-model.predict(X_test)
c0=model.predict(X_test)
probas=np.concatenate((c1,c0),axis=1)
#ROC curve 
skplt.metrics.plot_roc(y_test, probas)
#Precision-Recall curve 
skplt.metrics.plot_precision_recall(y_test, probas)

### BONUS QUESTION (optional)
Create a neural network architecture with one hidden layer (instead of 2; 13-32-1 architecture), train it, and compare its performance with the first NN (in terms of accuracy, precision, and recall).

You can copy the required cells from above and modify them.